![](images/logos/rosdevday.jpg)

# How to cross-compile ROS2 distro by taken VxWorks RTOS as an example

Even Open Robotics provides pre-built ROS 2 packages for multiple platforms, very often target software and hardware differ from the default one and a cross-compilation becomes a mandatory step:
* A different from Linux operating system e.g. VxWorks, QNX, eSol, etc. is deployed on the target hardware
* Target hardware (e.g. ARM aarch64) is different from the development host (e.g. Intel x86_64).
* Tuning target software for the footprint and performance (e.g. setting -mcpu=cortex-a72 -mfpu=neon-fp-armv8 when building for Raspberry Pi4).
* Separating ROS2 host tools (e.g. RViz) from the ROS2 target binaries.

This session will explain in detail of why cross-compilation is needed, and how to cross-compile ROS2 distro by taken VxWorks RTOS as an example. Step by step instructions will be given of how to setup a cross-compile development environment, to build and to deploy ROS2 binaries on the target. As a target QEMU Intel will be used on the Desktop PC.

## WHO AM I

<img align="left" src="images/headshot.jpg" width="200">
<center><strong>Andrei Kholodnyi</strong> | <strong>Principal Technologist</strong> | <strong>Technology Office</strong> | <strong>Wind River Systems</strong></center>

### <center>Focus </center>

<center> *  ROS2 Mobile Robotics, Dependability</center>
<center> *  Industrial, TSN, intelligent edge</center>
<center> *  ROS2 Open Source Community (real-time WG co-chair)</center>
<center> *  Products, Solutions; Partnerships & University Programs</center>

## Wind River software runs all these robots

![Wind River Software runs inside these robots](images/robots.jpg)

## What is VxWorks RTOS

<img align="left" src="images/vxworks.jpg" width="400">

<font size="5"> *  32/64 bits on ARM, Intel, MIPS, PowerPC, RISC-V</font>

<font size="5"> *  Proprietary real-time OS, POSIX PSE52</font>

<font size="5"> *  Kernel/user space separation, user space optional</font>

<font size="5"> *  C/C++11/14, possible to develop kernel C++ modules and user apps</font>
    
<font size="5"> *  Safety certifiable: DO-178, ISO 26262, IEC 61508</font>

<font size="5"> *  Toolchain LLVM 8, Dinkumware C/C++ libs</font>

<font size="5"> *  Proprietary build system</font>

<font size="5"> *  Kernel shell</font>

<font size="5"> *  Eclipse-based IDE, Windows/Linux hosts</font>

## What is a native compilation?

Let us look at this example of the compilation on the Intel PC running Linux Ubuntu. All artifacts (binaries, libs..) produced during a ROS2 build are supposed to run on the same platform.
The same can be done on RaspberryPi4 as well even it is a different hardware architecture.

![](images/native-compilation.jpg)

## What is a cross compilation?

What happens if we would substitute a desktop PC with an embedded target even with the same HW architecture (Intel x86_64) and would run a different OS, e.g. VxWorks RTOS.
As we can see many emebedded RTOS does not have a native development environment. They use host/target paradigm where a development happens on the host computer running Desktop OS e.g. Windows or Linux. And the development artifacts are deployed to the target.

Cross-compilation is a process of creating executable artifacts for a platform other than the one on which the cross-compilation toolchain is running. A cross-compilation toolchain is a set of chained tools used for this process 

![](images/cross-compilation.jpg)

## A native compilation – Hello, OS and HW Arch

Let us try to run this Hello, World code sample

```bash
$ cd examples
$ cat hello.c
```

```c
#include <stdio.h>
#include <sys/utsname.h>


int main() {
    struct utsname data;
    
    /* get name and information about current kernel */
    /* sysname[]; - Operating system name (e.g., "Linux") */
    /* machine[]; - Hardware identifier */
    uname(&data);
    printf("Hello, %s %s\n", data.sysname, data.machine);
    return 0;
}
```

```bash
$ which gcc
/usr/bin/gcc

$ gcc -print-multiarch
x86_64-linux-gnu
```

```bash
$ gcc -Wall hello.c -o hello -static
$ ./hello
Hello, Linux x86_64

$ file hello
hello: ELF 64-bit LSB executable, x86-64, version 1 (GNU/Linux), statically linked, BuildID[sha1]=e46d14d01d5ce7f9de399925757dc31afa564e58, for GNU/Linux 3.2.0, not stripped
```

## A cross compilation – Hello, OS and HW Arch

Let us do the same but with the cross-toolchain

```bash
$ source /opt/wrsdk-vxworks7-qemu/toolkit/wind_sdk_env.linux

$ env | grep WIND
WIND_SDK_COMPILER_PATH=/opt/wrsdk-vxworks7-qemu/toolkit/compilers/llvm-9.0.1.1/LINUX64/bin
WIND_SDK_HOST_TOOLS=/opt/wrsdk-vxworks7-qemu/toolkit/host_tools
WIND_SDK_LLVM_HOST_TYPE=LINUX64
WIND_SDK_LLVM_COMPILER=llvm-9.0.1.1
WIND_SDK_HOME=/opt/wrsdk-vxworks7-qemu
WIND_SDK_LLVM_PATH=/opt/wrsdk-vxworks7-qemu/toolkit/compilers/llvm-9.0.1.1
WIND_CC_SYSROOT=/opt/wrsdk-vxworks7-qemu/toolkit/include
WIND_SDK_VX7_HOST_TYPE=x86_64-linux
```

```bash
$ which $CC
/opt/wrsdk-vxworks7-qemu/toolkit/host_tools/x86_64-linux/bin/wr-cc

$ $CC -print-target-triple -c dummy.c
x86_64
```

```bash
$ cd examples
$ $CC -Wall hello.c -o hello -static
$ ./hello
Segmentation fault (core dumped)

$ file hello
hello: ELF 64-bit LSB executable, x86-64, version 1 (SYSV), statically linked, with debug_info, not stripped
```

Why it does not print VxWorks as an OS?

There are just few OSes defined by ELF format, see e.g. https://refspecs.linuxfoundation.org/elf/gabi4+/ch4.eheader.html

```bash
Name	          Value	Meaning
ELFOSABI_NONE        0	No extensions or unspecified
ELFOSABI_HPUX        1	Hewlett-Packard HP-UX
ELFOSABI_NETBSD      2	NetBSD
ELFOSABI_LINUX       3	Linux
ELFOSABI_SOLARIS     6	Sun Solaris
ELFOSABI_AIX         7	AIX
ELFOSABI_IRIX        8	IRIX
ELFOSABI_FREEBSD     9	FreeBSD
ELFOSABI_TRU64      10	Compaq TRU64 UNIX
ELFOSABI_MODESTO    11	Novell Modesto
ELFOSABI_OPENBSD    12	Open BSD
ELFOSABI_OPENVMS    13	Open VMS
ELFOSABI_NSK        14	Hewlett-Packard Non-Stop Kernel
 	            64-255	Architecture-specific value range
```

How can I figure out it a VxWorks binary? 

```bash
$ wr-nm hello | grep wrs
0000000000233288 r __wrs_eh_frame_end
000000000022c370 r __wrs_eh_frame_hdr_end
```

```bash
$ qemu-system-x86_64 -m 512M  -kernel $WIND_SDK_TOOLKIT/../bsps/itl_generic_2_0_2_1/boot/vxWorks -net nic -display none -serial stdio -monitor none -append "bootline:fs(0,0)host:vxWorks h=192.168.200.254 e=192.168.200.1 u=target pw=boot o=gei0" -usb -device usb-ehci,id=ehci  -device usb-storage,drive=fat32 -drive file=fat:ro:./,id=fat32,format=raw,if=none

Instantiating /ram0 as rawFs,  device = 0x1
Target Name: vxTarget
Instantiating /tmp as rawFs,  device = 0x10001


              VxWorks 7 SMP 64-bit

Copyright 1984-2020 Wind River Systems, Inc.

     Core Kernel version: 3.1.2.1
              Build date: Apr 21 2020 09:27:38
                   Board: x86 Processor (ACPI_BOOT_OP) SMP/SMT
               CPU Count: 1
          OS Memory Size: ~446MB
        ED&R Policy Mode: Deployed
     Debug Agent: Started (always)

Instantiating /ram as rawFs,  device = 0x20001
Formatting /ram for DOSFS
Instantiating /ram as rawFs, device = 0x20001
Formatting...Retrieved old volume params with %38 confidence:
Volume Parameters: FAT type: FAT32, sectors per cluster 0
  0 FAT copies, 0 clusters, 0 sectors per FAT
  Sectors reserved 0, hidden 0, FAT sectors 0
  Root dir entries 0, sysId (null)  , serial number 100000
  Label:"           " ...
Disk with 64 sectors of 512 bytes will be formatted with:
Volume Parameters: FAT type: FAT12, sectors per cluster 1
  2 FAT copies, 54 clusters, 1 sectors per FAT
  Sectors reserved 1, hidden 0, FAT sectors 2
  Root dir entries 112, sysId VXDOS12 , serial number 100000
  Label:"           " ...
OK.

 Adding 14058 symbols for standalone.

->
```

```bash
-> cmd
[vxWorks *]#
```

```bash
[vxWorks *]# devs
drv refs name
  1 [ 3] /
  2 [ 3] /bd0:1  ==>  /bd0a
  5 [ 3] /bd0a
  2 [ 3] /bin  ==>  /romfs/sysroot/bin
  2 [ 3] /boot  ==>  /romfs/sysroot/boot
  2 [ 3] /dev  ==>  /
  2 [ 3] /etc  ==>  /romfs/sysroot/etc
 10 [ 3] /fifos
 11 [ 3] /input/event
  2 [ 3] /lib  ==>  /romfs/sysroot/lib
  0 [ 3] /null
  5 [ 3] /ram
  6 [ 3] /ram0
 13 [ 3] /random
  9 [ 3] /romfs
  6 [ 3] /tmp
  3 [ 3] /ttyS0
  2 [ 3] /tyCo/0  ==>  /ttyS0
 13 [ 3] /urandom
  2 [ 3] /usr  ==>  /romfs/sysroot/usr
 12 [ 3] /zero
 15 [ 3] host:

[vxWorks *]# cd /bd0a
[vxWorks *]# ls
hello.c
hello
```

```bash
[vxWorks *]# cat hello.c
#include <stdio.h>
#include <sys/utsname.h>

int main() {
        struct utsname data;

        /* get name and information about current kernel */
        /* sysname[]; - Operating system name (e.g., "Linux") */
        /* machine[]; - Hardware identifier */

        uname(&data);
        printf("Hello, %s %s\n", data.sysname, data.machine);
        return 0;
}
```

```bash
[vxWorks *]# ./hello
Launching process './hello' ...
Process './hello' (process Id = 0xffff8000005bde80) launched.
Hello, VxWorks 7 x86 Processor (ACPI_BOOT_OP) SMP/SMT
```

## ROS2 distro structure

## Setup ROS2 for the cross-compilation

## toolchain.cmake and VxWorks.cmake

## cross-compile ROS2 

## Run ROS2 example using QEMU

## Conclusion